In [1]:
import pandas as pd
import numpy as np
import json

with open('plays.json') as data_file:    
    json_data = json.load(data_file)  


In [2]:
plays_json = pd.json_normalize(json_data)
plays_json.head()

,period,homeScore,scoringPlay,priority,statYardage,awayScore,wallclock,modified,id,text,...,expectedPoints.added,winProbability.before,winProbability.after,winProbability.added,start.shortDownDistanceText,start.possessionText,start.downDistanceText,scoringType.displayName,scoringType.name,scoringType.abbreviation
0,1,0,False,False,0,0,2020-12-21T19:34:38Z,2020-12-21T19:35Z,401256135101849902,Zach Williams kickoff for 60 yds,...,-2.823851,0.000100,0.075747,0.075647,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,False,False,-2,0,2020-12-21T19:36:09Z,2020-12-21T19:36Z,401256135101855901,Camerun Peoples run for a loss of 2 yards to t...,...,-1.372518,0.918380,0.898592,-0.019788,1st & 10,APP 35,1st & 10 at APP 35,NaN,NaN,NaN
2,1,0,False,False,0,0,2020-12-21T19:36:29Z,2020-12-21T19:36Z,401256135101858101,Zac Thomas pass incomplete to Jalen Virgil,...,-0.958074,0.898592,0.883065,-0.015527,2nd & 12,APP 33,2nd & 12 at APP 33,NaN,NaN,NaN
3,1,0,False,False,4,0,2020-12-21T19:37:31Z,2020-12-21T19:37Z,401256135101864601,Zac Thomas run for 4 yds to the AppSt 37,...,-0.664023,0.883065,0.879816,-0.003248,3rd & 12,APP 33,3rd & 12 at APP 33,NaN,NaN,NaN
4,1,0,False,False,0,0,2020-12-21T19:38:31Z,2020-12-21T19:38Z,401256135101867201,"Xavier Subotsch punt for 29 yds, downed at the...",...,-0.878193,0.879816,0.880597,0.000781,4th & 8,APP 37,4th & 8 at APP 37,NaN,NaN,NaN


In [3]:
plays_json.columns

Index(['period', 'homeScore', 'scoringPlay', 'priority', 'statYardage',
       'awayScore', 'wallclock', 'modified', 'id', 'text', 'playType',
       'start.distance', 'start.yardLine', 'start.team.id', 'start.down',
       'start.yardsToEndzone', 'start.posTeamTimeouts',
       'start.defTeamTimeouts', 'clock.displayValue', 'type.id', 'type.text',
       'type.abbreviation', 'end.shortDownDistanceText', 'end.possessionText',
       'end.downDistanceText', 'end.distance', 'end.yardLine', 'end.team.id',
       'end.down', 'end.yardsToEndzone', 'end.posTeamTimeouts',
       'end.defTeamTimeouts', 'expectedPoints.before', 'expectedPoints.after',
       'expectedPoints.added', 'winProbability.before', 'winProbability.after',
       'winProbability.added', 'start.shortDownDistanceText',
       'start.possessionText', 'start.downDistanceText',
       'scoringType.displayName', 'scoringType.name',
       'scoringType.abbreviation'],
      dtype='object')

In [54]:
##--Play type vectors------
scores_vec = [
"Blocked Punt Touchdown",
"Blocked Punt (Safety)",
"Punt (Safety)",
"Blocked Field Goal Touchdown",
"Missed Field Goal Return Touchdown",
"Fumble Recovery (Opponent) Touchdown",
"Fumble Return Touchdown",
"Interception Return Touchdown",
"Pass Interception Return Touchdown",
"Punt Touchdown",
"Punt Return Touchdown",
"Sack Touchdown",
"Uncategorized Touchdown",
"Defensive 2pt Conversion",
"Uncategorized",
"Two Point Rush",
"Safety",
"Penalty (Safety)",
"Punt Team Fumble Recovery Touchdown",
"Kickoff Team Fumble Recovery Touchdown",
"Kickoff (Safety)",
"Passing Touchdown",
"Rushing Touchdown",
"Field Goal Good",
"Pass Reception Touchdown",
"Fumble Recovery (Own) Touchdown"
]

defense_score_vec = [
"Blocked Punt Touchdown",
"Blocked Field Goal Touchdown",
"Missed Field Goal Return Touchdown",
"Punt Return Touchdown",
"Fumble Recovery (Opponent) Touchdown",
"Fumble Return Touchdown",
"Kickoff Return Touchdown",
"Defensive 2pt Conversion",
"Safety",
"Sack Touchdown",
"Interception Return Touchdown",
"Pass Interception Return Touchdown",
"Uncategorized Touchdown"
]
turnover_vec = [
  "Blocked Field Goal",
"Blocked Field Goal Touchdown",
"Blocked Punt",
"Blocked Punt Touchdown",
"Field Goal Missed",
"Missed Field Goal Return",
"Missed Field Goal Return Touchdown",
"Fumble Recovery (Opponent)",
"Fumble Recovery (Opponent) Touchdown",
"Fumble Return Touchdown",
"Defensive 2pt Conversion",
"Interception",
"Interception Return",
"Interception Return Touchdown",
"Pass Interception Return",
"Pass Interception Return Touchdown",
"Kickoff Team Fumble Recovery",
"Kickoff Team Fumble Recovery Touchdown",
"Punt Touchdown",
"Punt Return Touchdown",
"Sack Touchdown",
"Uncategorized Touchdown"
]
normalplay = [
"Rush",
"Pass",
"Pass Reception",
"Pass Incompletion",
"Pass Completion",
"Sack",
"Fumble Recovery (Own)"
]
penalty = [
'Penalty', 
'Penalty (Kickoff)',
'Penalty (Safety)'
]
offense_score_vec = [
"Passing Touchdown",
"Rushing Touchdown",
"Field Goal Good",
"Pass Reception Touchdown",
"Fumble Recovery (Own) Touchdown",
"Punt Touchdown", #<--- Punting Team recovers the return team fumble and scores
"Punt Team Fumble Recovery Touchdown",
"Kickoff Touchdown", #<--- Kickoff Team recovers the return team fumble and scores
"Kickoff Team Fumble Recovery Touchdown"
]
punt_vec = [
"Blocked Punt",
"Blocked Punt Touchdown",
"Blocked Punt (Safety)",
"Punt (Safety)",
"Punt",
"Punt Touchdown",
"Punt Team Fumble Recovery",
"Punt Team Fumble Recovery Touchdown",
"Punt Return Touchdown"
]
kickoff_vec = [
"Kickoff",
"Kickoff Return (Offense)",
"Kickoff Return Touchdown",
"Kickoff Touchdown",
"Kickoff Team Fumble Recovery",
"Kickoff Team Fumble Recovery Touchdown",
"Kickoff (Safety)",
"Penalty (Kickoff)"
]
int_vec = [
"Interception",
"Interception Return",
"Interception Return Touchdown",
"Pass Interception",
"Pass Interception Return",
"Pass Interception Return Touchdown"
]

def clean_pbp_dat(play_df):
    play_df.id = play_df.id.astype(int)
    play_df = play_df.sort_values(by="id", ascending=True)
    #-- Touchdowns----
    play_df["scoring_play"] = np.where(play_df["type.text"].isin(scores_vec), 1, 0)
    play_df["td_play"] = play_df.text.str.match("touchdown|for a TD", case=False, flags=0, na=False)
    play_df["touchdown"] = play_df["type.text"].str.match("touchdown", case=False, flags=0, na=False)
    play_df["safety"] = play_df["text"].str.match("safety", case=False, flags=0, na=False)

#       #-- Fumbles----
    play_df["fumble_vec"] = play_df["text"].str.match("fumble", case=False, flags=0, na=False)
    
#       #-- Kicks----
    play_df["kickoff_play"] = np.where(play_df["type.text"].isin(kickoff_vec), 1, 0)
    play_df["kickoff_tb"] = np.where(play_df["text"].str.match("touchback", case=False, flags=0, na=False) & play_df.kickoff_play == True, 1, 0)
    play_df["kickoff_onside"] = np.where(play_df["text"].str.match("on-side|onside|on side", case=False, flags=0, na=False) & play_df.kickoff_play == True, 1, 0)
    play_df["kickoff_oob"] = np.where(play_df["text"].str.match("out-of-bounds|out of bounds", case=False, flags=0, na=False) & play_df.kickoff_play == True, 1, 0)
    play_df["kickoff_fair_catch"] = np.where(play_df["text"].str.match("fair catch|fair caught", case=False, flags=0, na=False) & play_df.kickoff_play == True, 1, 0)
    play_df["kickoff_downed"] = np.where(play_df["text"].str.match("downed", case=False, flags=0, na=False) & play_df.kickoff_play == True, 1, 0)
    play_df["kick_play"] = play_df["text"].str.match("kick|kickoff", case=False, flags=0, na=False)
    play_df["kickoff_safety"] = np.where(~play_df["type.text"].isin(['Blocked Punt','Penalty']) & play_df["text"].str.match("kickoff", case=False, flags=0, na=False) & play_df.safety == True, 1, 0)
    
#       #-- Punts----
    play_df["punt"] = np.where(play_df["type.text"].isin(punt_vec), 1, 0)
    play_df["punt_play"] = play_df["text"].str.match("punt", case=False, flags=0, na=False)
    play_df["punt_tb"] = np.where(play_df["text"].str.match("touchback", case=False, flags=0, na=False) & play_df.punt == True, 1, 0)
    play_df["punt_oob"] = np.where(play_df["text"].str.match("out-of-bounds|out of bounds", case=False, flags=0, na=False) & play_df.punt == True, 1, 0)
    play_df["punt_fair_catch"] = np.where(play_df["text"].str.match("fair catch|fair caught", case=False, flags=0, na=False) & play_df.punt == True, 1, 0)
    play_df["punt_downed"] = np.where(play_df["text"].str.match("downed", case=False, flags=0, na=False) & play_df.punt == True, 1, 0)
    play_df["kickoff_safety"] = np.where(play_df["type.text"].isin(['Blocked Punt','Punt']) & play_df["text"].str.match("punt", case=False, flags=0, na=False) & play_df.safety == True, 1, 0)
    play_df["penalty_safety"] = np.where(play_df["type.text"].isin(['Penalty']) & play_df.safety == True, 1, 0)
    play_df["punt_blocked"] = np.where(play_df["text"].str.match("blocked", case=False, flags=0, na=False) & play_df.punt == True, 1, 0)

#       #-- Pass/Rush----
    play_df['rush'] = np.where(
        ((play_df["type.text"] == "Rush")
        | (play_df["type.text"] == "Rushing Touchdown")
        | (play_df["type.text"].isin(["Safety","Fumble Recovery (Opponent)","Fumble Recovery (Opponent) Touchdown", "Fumble Recovery (Own)", "Fumble Recovery (Own) Touchdown", "Fumble Return Touchdown"]) & play_df["text"].str.match("run for", case=False, flags=0, na=False))),
        1, 0
    )
    
    play_df['pass'] = np.where(
        (
        (play_df["type.text"].isin(["Pass Reception", "Pass Completion","Pass Touchdown","Sack","Pass","Inteception","Pass Interception Return", "Interception Return Touchdown","Pass Incompletion","Sack Touchdown"]))
        | ((play_df["type.text"] == "Safety") & play_df["text"].str.match("sacked", case=False, flags=0, na=False))
        | ((play_df["type.text"] == "Safety") & play_df["text"].str.match("pass complete", case=False, flags=0, na=False))
         | ((play_df["type.text"] == "Fumble Recovery (Own)") & play_df["text"].str.match("pass complete|pass incomplete|pass intercepted", case=False, flags=0, na=False))
         | ((play_df["type.text"] == "Fumble Recovery (Own)") & play_df["text"].str.match("sacked", case=False, flags=0, na=False))
         | ((play_df["type.text"] == "Fumble Recovery (Own) Touchdown") & play_df["text"].str.match("pass complete|pass incomplete|pass intercepted", case=False, flags=0, na=False))
          | ((play_df["type.text"] == "Fumble Recovery (Own) Touchdown") & play_df["text"].str.match("sacked", case=False, flags=0, na=False))
          | ((play_df["type.text"] == "Fumble Recovery (Opponent)") & play_df["text"].str.match("pass complete|pass incomplete|pass intercepted", case=False, flags=0, na=False))
           | ((play_df["type.text"] == "Fumble Recovery (Opponent)") & play_df["text"].str.match("sacked", case=False, flags=0, na=False))
           | ((play_df["type.text"] == "Fumble Recovery (Opponent) Touchdown") & play_df["text"].str.match("pass complete|pass incomplete", case=False, flags=0, na=False))
           | ((play_df["type.text"] == "Fumble Return Touchdown") & play_df["text"].str.match("pass complete|pass incomplete", case=False, flags=0, na=False))
            | ((play_df["type.text"] == "Fumble Return Touchdown") & play_df["text"].str.match("sacked", case=False, flags=0, na=False))
        ),
        1, 0
    )

      #-- Sacks----
    play_df['sack_vec'] = np.where(
        (
            (play_df["type.text"].isin(["Sack", "Sack Touchdown"]))
            | ((play_df["type.text"].isin(["Fumble Recovery (Own)", "Fumble Recovery (Own) Touchdown", "Fumble Recovery (Opponent)", "Fumble Recovery (Opponent) Touchdown", "Fumble Return Touchdown"]) 
               & (play_df["pass"] == 1)
               & (play_df["text"].str.match("sacked", case=False, flags=0, na=False))
              ))
        ), 1, 0)
    
    play_df['type.text'] = np.where(play_df["text"].str.match(" coin toss ", case=False, flags=0, na=False), "Coin Toss", play_df["type.text"])
    play_df['change_of_poss'] = np.where(play_df["start.team.id"] == play_df["start.team.id"].shift(-1), 0, 1)
    play_df['change_of_poss'] = np.where(play_df['change_of_poss'].isna(), 0, play_df['change_of_poss'])

      ## Fix Strip-Sacks to Fumbles----
#       play_type = ifelse(.data$fumble_vec == 1 & .data$pass == 1 & 
#                            .data$change_of_poss == 1 & .data$td_play == 0 & .data$down != 4 &
#                            !(.data$play_type %in% defense_score_vec),
#                          "Fumble Recovery (Opponent)", .data$play_type),
#       play_type = ifelse(.data$fumble_vec == 1 & .data$pass == 1 & 
#                            .data$change_of_poss == 1 & .data$td_play == 1, 
#                          "Fumble Recovery (Opponent) Touchdown", .data$play_type),
#       ## Fix rushes with fumbles and a change of possession to fumbles----
#       play_type = ifelse(.data$fumble_vec == 1 & .data$rush == 1 & 
#                            .data$change_of_poss == 1 & .data$td_play == 0 & 
#                            !(.data$play_type %in% defense_score_vec),
#                          "Fumble Recovery (Opponent)", .data$play_type),
#       play_type = ifelse(.data$fumble_vec == 1 & .data$rush == 1 & 
#                            .data$change_of_poss == 1 & .data$td_play == 1, 
#                          "Fumble Recovery (Opponent) Touchdown", .data$play_type),
#       ## Portion of touchdown check for plays where touchdown is not listed in the play_type--
#       td_check = ifelse(!str_detect(.data$play_type, "Touchdown"), 1, 0),
#       #-- Fix kickoff fumble return TDs ----
#       play_type = ifelse(.data$kickoff_play == 1 & .data$fumble_vec == 1 & 
#                            .data$td_play == 1 & .data$td_check == 1,
#                          paste0(.data$play_type, " Touchdown"), 
#                          .data$play_type),
#       #-- Fix punt return TDs ----
#       play_type = ifelse(.data$punt_play == 1 & .data$td_play == 1 & .data$td_check == 1,
#                          paste0(.data$play_type, " Touchdown"), 
#                          .data$play_type),
#       #-- Fix kick return TDs----
#       play_type = ifelse(.data$kickoff_play == 1 & .data$fumble_vec == 0 &
#                            .data$td_play == 1 & .data$td_check == 1,
#                          "Kickoff Return Touchdown", 
#                          .data$play_type),
#       #-- Fix rush/pass tds that aren't explicit----
#       play_type = ifelse(.data$td_play == 1 & .data$rush == 1 &
#                            .data$fumble_vec == 0 & .data$td_check == 1,
#                          "Rushing Touchdown", 
#                          .data$play_type),
#       play_type = ifelse(.data$td_play == 1 & .data$pass == 1 & .data$td_check == 1 &
#                            .data$fumble_vec == 0 & !(.data$play_type %in% int_vec),
#                          "Passing Touchdown", 
#                          .data$play_type),
#       play_type = ifelse(.data$pass == 1 & .data$play_type == "Pass Reception" &
#                            .data$yards_gained == .data$yards_to_goal &
#                            .data$fumble_vec == 0 & !(.data$play_type %in% int_vec),
#                          "Passing Touchdown", 
#                          .data$play_type),
#       play_type = ifelse(.data$play_type == "Blocked Field Goal" & 
#                            stringr::str_detect(.data$play_text, regex("for a TD", ignore_case = TRUE)),
#                          "Blocked Field Goal Touchdown", 
#                          .data$play_type),
#       #-- Fix duplicated TD play_type labels----
#       play_type = ifelse(.data$play_type == "Punt Touchdown Touchdown", "Punt Touchdown", .data$play_type),
#       play_type = ifelse(.data$play_type == "Fumble Return Touchdown Touchdown", "Fumble Return Touchdown", .data$play_type),
#       play_type = ifelse(.data$play_type == "Rushing Touchdown Touchdown", "Rushing Touchdown", .data$play_type),
#       play_type = ifelse(.data$play_type == "Uncategorized Touchdown Touchdown", "Uncategorized Touchdown", .data$play_type),
#       #-- Fix Pass Interception Return TD play_type labels----
#       play_type = ifelse(stringr::str_detect(.data$play_text, "pass intercepted for a TD") & !is.na(.data$play_text),
#                          "Interception Return Touchdown", .data$play_type),
#       #-- Fix Sack/Fumbles Touchdown play_type labels----
#       play_type = ifelse(stringr::str_detect(.data$play_text, regex("sacked", ignore_case = TRUE)) & 
#                            stringr::str_detect(.data$play_text, regex("fumbled", ignore_case = TRUE)) &
#                            stringr::str_detect(.data$play_text, regex("TD",ignore_case = TRUE)) &
#                            !is.na(.data$play_text), 
#                          "Fumble Recovery (Opponent) Touchdown", .data$play_type),
#       #-- Fix generic pass plays ----
#       ##-- first one looks for complete pass
#       play_type = ifelse(.data$play_type == "Pass" & str_detect(.data$play_text,"pass complete"),
#                          "Pass Completion", .data$play_type),
#       ##-- second one looks for incomplete pass
#       play_type = ifelse(.data$play_type == "Pass" & str_detect(.data$play_text,"pass incomplete"),
#                          "Pass Incompletion", .data$play_type),
#       ##-- third one looks for interceptions 
#       play_type = ifelse(.data$play_type == "Pass" & str_detect(.data$play_text,"pass intercepted"),
#                          "Pass Interception", .data$play_type),
#       ##-- fourth one looks for sacked
#       play_type = ifelse(.data$play_type == "Pass" & str_detect(.data$play_text,"sacked"), "Sack", .data$play_type),
#       ##-- fifth one play type is Passing Touchdown, but its intercepted 
#       play_type = ifelse(.data$play_type == "Passing Touchdown" & str_detect(.data$play_text,"pass intercepted for a TD"),
#                          "Interception Return Touchdown", .data$play_type),
#       #--- Moving non-Touchdown pass interceptions to one play_type: "Interception Return" -----
#       play_type = ifelse(.data$play_type == "Interception", "Interception Return", .data$play_type),
#       play_type = ifelse(.data$play_type == "Pass Interception", "Interception Return", .data$play_type),
#       play_type = ifelse(.data$play_type == "Pass Interception Return", "Interception Return", .data$play_type),
#       #--- Moving Kickoff/Punt Touchdowns without fumbles to Kickoff/Punt Return Touchdown
#       play_type = ifelse(.data$play_type == "Kickoff Touchdown" & .data$fumble_vec == 0, 
#                          "Kickoff Return Touchdown", .data$play_type),
#       play_type = ifelse(.data$play_type %in% c("Kickoff", "Kickoff Return (Offense)") & 
#                            .data$fumble_vec == 1 & .data$change_of_pos_team == 1, 
#                          "Kickoff Team Fumble Recovery", .data$play_type),
#       play_type = ifelse(.data$play_type == "Punt Touchdown" & 
#                            (.data$fumble_vec == 0 | (.data$fumble_vec == 1 & .data$game_id == 401112100)),
#                          "Punt Return Touchdown", .data$play_type),
#       play_type = ifelse(.data$play_type == "Punt" & .data$fumble_vec == 1 & .data$change_of_poss == 0,
#                          "Punt Team Fumble Recovery", .data$play_type),
#       play_type = ifelse(.data$play_type == "Punt Touchdown", "Punt Team Fumble Recovery Touchdown", .data$play_type),
#       play_type = ifelse(.data$play_type == "Kickoff Touchdown", "Kickoff Team Fumble Recovery Touchdown", .data$play_type),
#       play_type = ifelse(.data$play_type == "Fumble Return Touchdown" & (.data$pass == 1|.data$rush == 1), 
#                          "Fumble Recovery (Opponent) Touchdown", .data$play_type),
#       #--- Safeties (kickoff, punt, penalty) ----
#       play_type = ifelse(.data$play_type %in% c("Pass Reception", "Rush", "Rushing Touchdown") & 
#                            (.data$pass == 1|.data$rush == 1) & .data$safety == 1, 
#                          "Safety", .data$play_type),
#       play_type = ifelse(.data$kickoff_safety == 1, "Kickoff (Safety)", .data$play_type),
#       play_type = ifelse(.data$punt_safety == 1,  paste0(.data$play_type, " (Safety)"), .data$play_type),
#       play_type = ifelse(.data$penalty_safety == 1,  paste0(.data$play_type, " (Safety)"), .data$play_type),
#       id_play = ifelse(.data$id_play == 400852742102997104 & .data$play_type == "Kickoff", 400852742102997106, .data$id_play),
#       id_play = ifelse(.data$id_play == 400852742102997106 & .data$play_type == "Defensive 2pt Conversion", 400852742102997104, .data$id_play),
#       #--- Sacks ----
#       sack = ifelse((.data$play_type %in% c("Sack") |
#                        (.data$play_type %in% c("Fumble Recovery (Own)", 
#                                                "Fumble Recovery (Own) Touchdown",
#                                                "Fumble Recovery (Opponent)", 
#                                                "Fumble Recovery (Opponent) Touchdown") & 
#                           .data$pass == 1 & stringr::str_detect(.data$play_text, "sacked")) |
#                        (.data$play_type == "Safety" & stringr::str_detect(.data$play_text, regex('sacked', ignore_case = TRUE)))) & 
#                       !is.na(.data$play_text), 1, 0),
#       #--- Interceptions ------
#       int = ifelse(.data$play_type %in% c("Interception Return", "Interception Return Touchdown"), 1, 0),
#       int_td = ifelse(.data$play_type %in% c("Interception Return Touchdown"), 1, 0),
#       #--- Pass Completions, Attempts and Targets -------
#       completion = ifelse(.data$play_type %in% c("Pass Reception","Pass Completion", "Passing Touchdown")|
#                           ((.data$play_type %in% c("Fumble Recovery (Own)",
#                                                    "Fumble Recovery (Own) Touchdown",
#                                                    "Fumble Recovery (Opponent)",
#                                                    "Fumble Recovery (Opponent) Touchdown") & .data$pass == 1 &
#                           !stringr::str_detect(.data$play_text,"sacked"))), 1, 0),
#       pass_attempt = ifelse(.data$play_type %in% c("Pass Reception",
#                                                    "Pass Completion",
#                                                    "Passing Touchdown", 
#                                                    "Pass Incompletion",
#                                                    "Interception Return", 
#                                                    "Interception Return Touchdown") |
#                             ((.data$play_type %in% c("Fumble Recovery (Own)",
#                                                      "Fumble Recovery (Own) Touchdown",
#                                                      "Fumble Recovery (Opponent)",
#                                                      "Fumble Recovery (Opponent) Touchdown") & .data$pass == 1 &
#                             !stringr::str_detect(.data$play_text,"sacked"))), 1, 0),
#       target = ifelse(.data$play_type %in% c("Pass Reception",
#                                              "Pass Completion",
#                                              "Passing Touchdown",
#                                              "Pass Incompletion") |
#                       ((.data$play_type %in% c("Fumble Recovery (Own)",
#                                                "Fumble Recovery (Own) Touchdown",
#                                                "Fumble Recovery (Opponent)",
#                                                "Fumble Recovery (Opponent) Touchdown") & .data$pass == 1 &
#                             !stringr::str_detect(.data$play_text,"sacked"))), 1, 0),
#       #--- Pass/Rush TDs ------
#       pass_td = ifelse(.data$play_type %in% c("Passing Touchdown"), 1, 0),
#       rush_td = ifelse(.data$play_type %in% c("Rushing Touchdown"), 1, 0),
#       #-- Change of possession via turnover
#       turnover_vec = ifelse(.data$play_type %in% turnover_vec, 1, 0),
#       offense_score_play = ifelse(.data$play_type %in% offense_score_vec, 1, 0),
#       defense_score_play = ifelse(.data$play_type %in% defense_score_vec, 1, 0),
#       downs_turnover = ifelse((.data$play_type %in% normalplay) & 
#                               (.data$yards_gained < .data$distance) & (.data$down == 4) &
#                               !(.data$penalty_1st_conv), 1, 0),
#       #-- Touchdowns----
#       scoring_play = ifelse(.data$play_type %in% scores_vec, 1, 0),
#       yds_punted = ifelse(.data$punt == 1, as.numeric(stringr::str_extract(
#         stringi::stri_extract_first_regex(.data$play_text, '(?<= punt for)[^,]+'),
#         "\\d+"
#       )), NA_real_),
#       yds_punt_gained = ifelse(.data$punt == 1, .data$yards_gained, NA_real_),
#       fg_inds = ifelse(stringr::str_detect(.data$play_type, "Field Goal"), 1, 0),
#       fg_made = ifelse(.data$play_type == "Field Goal Good", TRUE, FALSE),
#       yds_fg = ifelse(.data$fg_inds == 1, as.numeric(
#         stringr::str_remove(stringr::str_extract(.data$play_text, 
#                                                  regex('\\d{0,2} Yd FG|\\d{0,2} Yd Field|\\d{0,2} Yard Field', ignore_case = TRUE)), 
#                             regex("yd FG|yd field|yard field", ignore_case = TRUE))), NA),
#       yards_to_goal = ifelse(.data$fg_inds == 1 & !is.na(.data$yds_fg), .data$yds_fg-17, .data$yards_to_goal),
#       yards_to_goal = ifelse(.data$id_play == "401112476101977728", 16, .data$yards_to_goal),
#       yards_to_goal = ifelse(.data$id_play == "401112476104999424", 36, .data$yards_to_goal),
#       pos_unit = dplyr::case_when(
#         .data$punt == 1 ~ "Punt Offense",
#         .data$kickoff_play == 1 ~ "Kickoff Return",
#         .data$fg_inds == 1 ~ "Field Goal Offense",
#         .data$play_type == "Defensive 2pt Conversion" ~ "Offense",
#         TRUE ~ "Offense"
#       ),
#       def_pos_unit = dplyr::case_when(
#         .data$punt == 1 ~ "Punt Return",
#         .data$kickoff_play == 1 ~ "Kickoff Defense",
#         .data$fg_inds == 1 ~ "Field Goal Defense",
#         .data$play_type == "Defensive 2pt Conversion" ~ "Defense",
#         TRUE ~ "Defense"
#       ),
#       #--- Lags/Leads play type ----
    play_df['lag_play_type3'] = play_df['lag_play_type3'].shift(-3)
    play_df['lag_play_type2'] = play_df['lag_play_type3'].shift(-3)
    play_df['lag_play_type1'] = play_df['lag_play_type3'].shift(-3)
      lag_play_type2 = dplyr::lag(.data$play_type, 2),
      lag_play_type = dplyr::lag(.data$play_type, 1),
      lead_play_type = dplyr::lead(.data$play_type, 1),
      lead_play_type2 = dplyr::lead(.data$play_type, 2),
      lead_play_type3 = dplyr::lead(.data$play_type, 3)
#     )
    return play_df

In [55]:
clean_pbp_dat(plays_json)

,period,homeScore,scoringPlay,priority,statYardage,awayScore,wallclock,modified,id,text,...,punt_tb,punt_oob,punt_fair_catch,punt_downed,penalty_safety,punt_blocked,rush,pass,sack_vec,change_of_poss
0,1,0,False,False,0,0,2020-12-21T19:34:38Z,2020-12-21T19:35Z,401256135101849902,Zach Williams kickoff for 60 yds,...,0,0,0,0,0,0,0,0,0,1
1,1,0,False,False,-2,0,2020-12-21T19:36:09Z,2020-12-21T19:36Z,401256135101855901,Camerun Peoples run for a loss of 2 yards to t...,...,0,0,0,0,0,0,1,0,0,0
2,1,0,False,False,0,0,2020-12-21T19:36:29Z,2020-12-21T19:36Z,401256135101858101,Zac Thomas pass incomplete to Jalen Virgil,...,0,0,0,0,0,0,0,1,0,0
3,1,0,False,False,4,0,2020-12-21T19:37:31Z,2020-12-21T19:37Z,401256135101864601,Zac Thomas run for 4 yds to the AppSt 37,...,0,0,0,0,0,0,1,0,0,0
4,1,0,False,False,0,0,2020-12-21T19:38:31Z,2020-12-21T19:38Z,401256135101867201,"Xavier Subotsch punt for 29 yds, downed at the...",...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,4,49,False,False,16,23,2020-12-23T03:23:40Z,2020-12-23T03:23Z,401256137104979401,Zach Wilson pass complete to Dax Milne for 16 ...,...,0,0,0,0,0,0,0,1,0,0
375,4,49,False,False,0,23,2020-12-23T03:24:04Z,2020-12-23T03:24Z,401256137104986501,"Timeout UCF, clock 01:34",...,0,0,0,0,0,0,0,0,0,0
376,4,49,False,False,-2,23,2020-12-23T03:28:13Z,2020-12-23T03:28Z,401256137104986901,TEAM run for a loss of 2 yards to the UCF 6,...,0,0,0,0,0,0,1,0,0,0
377,4,49,False,False,-1,23,2020-12-23T03:28:45Z,2020-12-23T03:28Z,401256137104994401,TEAM run for a loss of 1 yard to the UCF 7,...,0,0,0,0,0,0,1,0,0,0
